In [1]:
import pandas as pd
import math as mt

###ID3 method
def Ent(name,df):
    ent=0
    count=df[name].value_counts()
    tot=df[name].size
    for i in count:
        p=i/tot
        ent+= mt.log2(p)*p
    return -ent  
def Gain(name_main,name_sub,df):
    
    count=df[name_sub].value_counts()
    tot=df[name_sub].size
    summ=0
    for i in count.index:
        summ+=count[i]*Ent(name_main,df[df[name_sub]==i])
    return Ent(name_main,df)-summ/tot

### CART method
def Gini(name, df):
    gini=0
    count=df[name].value_counts()
    tot=df[name].size
    for i in count:
        p=i/tot
        gini+= p**2
    return 1-gini

def Gini_index(name_main,name_sub,df):
    count=df[name_sub].value_counts()
    tot=df[name_sub].size
    summ=0
    for i in count.index:
        summ+=count[i]*Gini(name_main,df[df[name_sub]==i])
    return summ/tot

In [4]:
### First let's see the data 
df=pd.read_excel(r'C:\Users\Admin\Downloads\3.xlsx')
target='haogua'
print(df[target].value_counts().index)
print(df)
### the data have 6 characteristics of a watermelon
### and 1 target variable of whether the water melon taste good or bad

Index(['坏瓜', '好瓜'], dtype='object')
    色泽  根蒂  敲声  纹理  脐部  触感 haogua
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑     好瓜
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑     好瓜
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑     好瓜
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑     好瓜
4   浅白  蜷缩  浊响  清晰  凹陷  硬滑     好瓜
5   青绿  稍蜷  浊响  清晰  稍凹  软粘     好瓜
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘     好瓜
7   乌黑  稍蜷  浊响  清晰  稍凹  硬滑     好瓜
8   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑     坏瓜
9   青绿  硬挺  清脆  清晰  平坦  软粘     坏瓜
10  浅白  硬挺  清脆  模糊  平坦  硬滑     坏瓜
11  浅白  蜷缩  浊响  模糊  平坦  软粘     坏瓜
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑     坏瓜
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑     坏瓜
14  乌黑  稍蜷  浊响  清晰  稍凹  软粘     坏瓜
15  浅白  蜷缩  浊响  模糊  平坦  硬滑     坏瓜
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑     坏瓜


In [8]:
###recursive output, desicion tree is in the format of dictionary
def output(df,f):
    Gs={}
    size=df[target].size
    ###end recursion sitution 1, only one class left
    if df.columns.size==2:
        for col in df.columns:
            if col!=target:
                result={}
                for classs in df[col].unique():
                    count=df[df[col]==classs][target].value_counts()
                    x=0
                    for i in count.index:
                        if x==0:
                            big=count[i]
                            bigname=i
                        else:
                            if big<count[i]:
                                big=count[i]
                                bigname=i
                        x+=1
                    colname=col
                    result[classs]=bigname
                
                return {colname:result}            
    ###end recursion sitution 2, data left only contain either good or bad melon
    for i in df[target].value_counts().index:
        if df[target].value_counts()[i]==size:
            return i
    ###recurson condition: output argmax(objective function (Gain/ Gini))
    for col in df.columns:
        if col!=target:
            Gs[col]=f(target,col,df)
    x=0
    for i in Gs:
        if x==0:
            big=Gs[i]
            bigname=i
        else:
            if big<Gs[i]:
                big=Gs[i]
                bigname=i             
        x+=1

    return {bigname:{c:output(df.drop(columns=bigname)[df[bigname]==c],f) for c in df[bigname].unique()}}

In [9]:
### ID3 method
output(df,Gain)

{'纹理': {'清晰': {'根蒂': {'蜷缩': '好瓜',
    '稍蜷': {'色泽': {'青绿': '好瓜', '乌黑': {'触感': {'硬滑': '好瓜', '软粘': '坏瓜'}}}},
    '硬挺': '坏瓜'}},
  '稍糊': {'触感': {'软粘': '好瓜', '硬滑': '坏瓜'}},
  '模糊': '坏瓜'}}

In [10]:
### CART method
output(df,Gini_index)

{'触感': {'硬滑': {'敲声': {'浊响': {'根蒂': {'蜷缩': {'色泽': {'青绿': '好瓜',
        '乌黑': '好瓜',
        '浅白': {'纹理': {'清晰': {'脐部': {'凹陷': '好瓜'}},
          '模糊': {'脐部': {'平坦': '坏瓜'}}}}}},
      '稍蜷': {'色泽': {'乌黑': '好瓜', '青绿': '坏瓜'}}}},
    '沉闷': {'色泽': {'乌黑': {'根蒂': {'蜷缩': '好瓜', '稍蜷': '坏瓜'}},
      '青绿': {'根蒂': {'蜷缩': {'纹理': {'清晰': {'脐部': {'凹陷': '好瓜'}},
          '稍糊': {'脐部': {'稍凹': '坏瓜'}}}}}},
      '浅白': '坏瓜'}},
    '清脆': '坏瓜'}},
  '软粘': {'色泽': {'青绿': {'纹理': {'清晰': {'根蒂': {'稍蜷': '好瓜', '硬挺': '坏瓜'}}}},
    '乌黑': {'根蒂': {'稍蜷': {'敲声': {'浊响': {'脐部': {'稍凹': {'纹理': {'稍糊': '好瓜',
            '清晰': '坏瓜'}}}}}}}},
    '浅白': '坏瓜'}}}}

In [ ]:
import pydot
import graphviz
def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, dict):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)

graph = pydot.Dot(graph_type='graph')
visit(output(df,Gain))
graph.write_png(r'C:\Users\Admin\Downloads\example1_graph.png')